# Dados

In [ ]:
import requests
from io import StringIO

# parametros da request (github info)
owner = 'danplevs'
token = 'd9d7f4c1c64d6ec82b7c086c9720698d21ab9f48'
repo = 'geohmount'
path = 'geohmount-plotly/aerossol-marinho/chuva_nacl.csv'

# enviando o request
r = requests.get(f'https://api.github.com/repos/{owner}/{repo}/contents/{path}',
    headers={
        'accept': 'application/vnd.github.v3.raw',
        'authorization': f'token {token}'
            }
    )

# convertendo a string em objeto stringIO
string_io_obj = StringIO(r.text)

In [ ]:
import pandas as pd

nacl = pd.read_csv(string_io_obj, sep=';')
nacl.head()

In [ ]:
nacl = nacl.drop(0)

In [ ]:
nacl['Ponto'] = nacl['Amostras'].str.slice(start=0, stop=2)
nacl.head()

In [ ]:
sb = nacl.query("Ponto == 'SB'")
ps = nacl.query("Ponto == 'PS'")
bm = nacl.query("Ponto == 'BM'")
sm = nacl.query("Ponto == 'SM'")

In [ ]:
sm = sm.drop(44)

# Gráfico

In [ ]:
config = {'displayModeBar': True, 'toImageButtonOptions': {'height': None, 'width': None}}

In [ ]:
import base64

# set a local image as the background
img = 'C:\\Users\\daniel\\Documents\\Github\\geohmount\\logos-png\\GEOHMOUNT-Logo-Cinzas.png'
geohmount_logo = base64.b64encode(open(img, 'rb').read())

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    name='SB',
    x=sb.iloc[:, 1], y=sb.iloc[:, 2],
    marker=dict(color='rgb(112, 173, 71)', symbol='square', size=10),
    mode='markers'
))

fig.add_trace(go.Scatter(
    name='PS',
    x=ps.iloc[:, 1], y=ps.iloc[:, 2],
    marker=dict(color='rgb(91, 155, 213)', symbol='triangle-up', size=10),
    mode='markers'
))

fig.add_trace(go.Scatter(
    name='BM',
    x=bm.iloc[:, 1], y=bm.iloc[:, 2],
    marker=dict(color='rgb(255, 192, 0)', symbol='circle', size=10),
    mode='markers'
))

fig.add_trace(go.Scatter(
    name='SM',
    x=sm.iloc[:, 1], y=sm.iloc[:, 2],
    marker=dict(color='rgb(37, 94, 145)', symbol='diamond', size=10),
    mode='markers'
))

fig.add_trace(go.Scatter(
    name='Água do mar',
    x=[0, 120], y=[0, 139.8],
    mode='lines',
    line=dict(color='darkgreen', dash='dot', width=2)
))

fig.add_layout_image(
    dict(
        source=f'data:image/png;base64,{geohmount_logo.decode()}',
        xref="paper",
        yref="paper",
        x=0.18,
        y=0.87,
        sizex=0.65,
        sizey=0.65,
        sizing="contain",
        opacity=0.13,
        layer="below")
)

fig.update_layout(template='plotly_white', width=1200, height=800, legend=dict(font_size=16, orientation='h', xanchor='center', yanchor='bottom', x=0.5, y=-0.2),
updatemenus=[
    dict(
        buttons=list([
            dict(label='Linear', method='relayout', args=[{'yaxis.type': 'linear', 'yaxis.tickvals': (0, 20, 40, 60, 80, 100, 120, 140), 'xaxis.type': 'linear', 'xaxis.tickvals': (0, 20, 40, 60, 80, 100, 120)}]),
            dict(label='Log', method='relayout', args=[{'yaxis.type': 'log', 'yaxis.tickvals': (0.1, 1, 10, 100), 'xaxis.type': 'log', 'xaxis.tickvals': (0.1, 1, 10, 100)}])
        ])
    )
],
annotations=[
    dict(text='<b>Escala</b>', showarrow=False, xref='paper', yref='paper', x=-0.111, y=1.05, font_size=14)
]
)
fig.update_traces(hovertemplate='<b>Na<sup>+</sup>: %{x:.2f}<br>'+'<b>Cl<sup>-</sup>: %{y:.2f}')
fig.update_yaxes(title=dict(text='<b>Cl<sup>-</sup> (mol ha<sup>-1</sup>)</b>', font_size=16), tickfont=dict(size=16))
fig.update_xaxes(title=dict(text='<b>Na<sup>+</sup> (mol ha<sup>-1</sup>)</b>', font_size=16), tickfont=dict(size=16))
fig.show(config=config)
#fig.write_html('nacl.html', include_plotlyjs='cdn', config=config)